In [1]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=2bf955bf91d65892252b32c01c048701d32c0e83e21471839a7e30514bd3508c
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [3]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix,classification_report

from collections import Counter #for Smote, 
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
#import datafiles
trainfile = r'/gdrive/My Drive/DataMining/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)
testfile = r'/gdrive/My Drive/DataMining/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head())

(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [5]:
#Separate Train data and test data
X_train = train_data.drop(columns="QuoteConversion_Flag").copy()
y_train = train_data["QuoteConversion_Flag"].copy()
X_test = test_data.drop(columns="GeographicField64").copy()
y_test = test_data["GeographicField64"].copy()

#Select just Target Column
print(X_train.shape)
print(X_test.head()) 


(65000, 595)
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]


In [6]:
#Decision Tree Model

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':clf_predict})

pred.to_csv("/gdrive/My Drive/DT1.csv",index = False)

res=pd.read_csv('/gdrive/My Drive/DT1.csv')
res.head()

parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':clf_predict})

pred.to_csv("/gdrive/My Drive/DT2.csv",index = False)
res=pd.read_csv('/gdrive/My Drive/DT2.csv')
res.head()


{'min_samples_split': 70, 'max_depth': 5}


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [ ]:
#Random Forest Classifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict = rfc.predict(X_test)


pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':rfc_predict})
pred.to_csv("/gdrive/My Drive/RF1.csv")

res=pd.read_csv('/gdrive/My Drive/RF1.csv')
res.head()

parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)

pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':rfc_predict})
pred.to_csv("/gdrive/My Drive/RF2.csv")
res=pd.read_csv('/gdrive/My Drive/RF2.csv')
res.head()

In [ ]:
#Gradient Boosting Classifier
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)

pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':abc_predict})
pred.to_csv("/gdrive/My Drive/GB1.csv")

res=pd.read_csv('/gdrive/My Drive/GB1.csv')
res.head()

#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)


pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':abc_predict})
pred.to_csv("/gdrive/My Drive/GB2.csv")
res=pd.read_csv('/gdrive/My Drive/GB2.csv')
res.head()

#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())


In [ ]:
#MultiLayer Perceptron
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
mlp_predict=mlp.predict(X_test)
pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':mlp_predict})
pred.to_csv("/gdrive/My Drive/MLP.csv")
res=pd.read_csv('/gdrive/My Drive/MLP.csv')
res.head()

In [ ]:
#SVC Linear

svc = LinearSVC()
svc.fit(X_train, y_train)
svc_predict=svc.predict(X_test)


pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':svc_predict})

pred.to_csv("/gdrive/My Drive/SVC.csv")
res=pd.read_csv('/gdrive/My Drive/SVC.csv')

res.head()

In [ ]:
#KNN Classifier
knc = KNeighborsClassifier()
knc.fit(X_train, y_train)
knc_predict = knc.predict(X_test)
print(accuracy_score(y_test,knc_predict))


pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':knc_predict})

pred.to_csv("/gdrive/My Drive/KNN.csv")
res=pd.read_csv('/gdrive/My Drive/KNN.csv')res.head()

In [8]:
#SMOTE
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [9]:
#Stacking Models

models = [ DecisionTreeClassifier(),GradientBoostingClassifier(), MLPClassifier(),KNeighborsClassifier(),LinearSVC() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.89968145]
    fold  1:  [0.90256358]
    fold  2:  [0.90114780]
    fold  3:  [0.90402508]
    ----
    MEAN:     [0.90185448] + [0.00161522]
    FULL:     [0.90185445]

model  1:     [GradientBoostingClassifier]
    fold  0:  [0.93558174]
    fold  1:  [0.93254791]
    fold  2:  [0.93563230]
    fold  3:  [0.93542678]
    ----
    MEAN:     [0.93479718] + [0.00130082]
    FULL:     [0.93479717]

model  2:     [MLPClassifier]
    fold  0:  [0.82686960]
    fold  1:  [0.79734034]
    fold  2:  [0.83612277]
    fold  3:  [0.79940332]
    ----
    MEAN:     [0.81493401] + [0.01689794]
    FULL:     [0.81493420]

model  3:     [KNeighborsClassifier]
    fold  0:  [0.70000506]
    fold  1:  [0.69636446]
    fold  2:  [0.69737574]
    fold  3:  [0.69614684]
    ----
    MEAN:     [0.69747303] + [0.00153365]
   

In [10]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)


In [11]:

pred=pd.DataFrame({'QuoteNumber':test_data.QuoteNumber,'QuoteConversion_Flag':y_pred})
pred.to_csv("/gdrive/My Drive/finalsub.csv",index = False)
res=pd.read_csv('/gdrive/My Drive/finalsub.csv')
res.head()


,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
